## Forward Indexing
- 문서에 대한 단어들의 구조

In [2]:
import re 
import nltk

In [3]:
import glob
fileList = glob.glob('doc/*')

In [5]:
from konlpy.tag import Kkma

In [6]:
uniqueNouns = []
for file in fileList:
    
    with open(file, 'r') as fp:
        
        content = fp.read()
        content = re.sub(r'[\s]{3,}',"", content)
        
    for sentence in nltk.sent_tokenize(content):
        nouns = []
        
        for word in nltk.word_tokenize(sentence):
            nouns.extend(Kkma().nouns(word))
            nouns = list(set(nouns))
            
        uniqueNouns.extend(nouns)
        uniqueNouns = list(set(uniqueNouns))

print(len(uniqueNouns))

1422


In [7]:
docNouns = {}
maxCount = {} # 각 문서 안에서 제일 많이 나온 단어의 수! 와 문서 이름! 만을 저장한다. 

for file in fileList:
    docid = file[-21:-4]
    
    with open(file, 'r') as fp:
        maxCount[docid] = 0
        content = fp.read()
        content = re.sub(r'[\s]{3,}',"", content)
      
    nouns = {}
    for sentence in nltk.sent_tokenize(content):        
        for word in nltk.word_tokenize(sentence):
            for w in Kkma().nouns(word):
                if w in nouns.keys():
                    nouns[w] += 1 # 있으면 벡터 값을 더한다. 
                else:
                    nouns[w] = 1 # 없으면 새로운 갑 추가 
                
                if maxCount[docid] < nouns[w]:
                    maxCount[docid] = nouns[w]
    docNouns[docid] = nouns        

In [8]:
for k,v in docNouns.items():
    print("{0} - {1}".format(k, len(v))) # 각 문서에서 나온 단어와 그 단어의 빈도수를 만든 딕셔너리 , 각 단어와 빈도 수 또한 딕셔너리 

20180801141549383 - 223
20180801144439453 - 109
20180801142859853 - 128
20180801143528088 - 142
20180801144119329 - 214
20180801145118652 - 235
20180801143420047 - 167
20180801141712421 - 123
20180801141138247 - 96
20180801145951917 - 232
20180801142952883 - 192
20180801144437449 - 0


In [9]:
for k,v in docNouns.items():
    print("{0} - {1}/{2}".format(k, len(v), maxCount[k])) # 각 문서에서 제일 많이 나온 단어의 수 

20180801141549383 - 223/18
20180801144439453 - 109/6
20180801142859853 - 128/10
20180801143528088 - 142/10
20180801144119329 - 214/7
20180801145118652 - 235/20
20180801143420047 - 167/12
20180801141712421 - 123/7
20180801141138247 - 96/5
20180801145951917 - 232/10
20180801142952883 - 192/8
20180801144437449 - 0/0


## Inverted Document
- 책 뒤에 있는 단어별로 나온 index 
- 단어를 통한 문서의 구조 확인
- C 로 하게 된다면 file 포인터를 통해서 하기 때문에 활용성이 굉장히 높아진다.

In [10]:
invertedIndex = {}

for noun in uniqueNouns:
    invertedIndex[noun] = []
    
    for k, nouns in docNouns.items():
        if noun in nouns:
            invertedIndex[noun].append(k)

In [11]:
for k, v in invertedIndex.items():
    print("{0} - {1}".format(k, invertedIndex[k]))

0.123 - ['20180801143528088']
종합적 - ['20180801141549383']
실적 - ['20180801141549383', '20180801145118652']
동계시즌 - ['20180801141549383']
하다 - ['20180801142952883']
공명 - ['20180801141712421']
마무리 - ['20180801141549383']
필요 - ['20180801144119329', '20180801143420047', '20180801142952883']
태 - ['20180801141549383']
분석 - ['20180801141549383', '20180801145951917']
운영 - ['20180801144119329']
출시 - ['20180801145118652']
1일 - ['20180801141549383', '20180801142859853', '20180801143528088', '20180801144119329', '20180801145118652', '20180801143420047', '20180801141712421', '20180801141138247', '20180801145951917']
심장병 - ['20180801143528088']
제천구조대 - ['20180801144119329']
동풍 - ['20180801145951917']
촉구 - ['20180801142952883']
접수 - ['20180801144119329']
계획 - ['20180801141549383', '20180801143420047']
중인 - ['20180801141549383', '20180801143420047']
도달 - ['20180801142952883']
심 - ['20180801141712421']
정치 - ['20180801141712421']
소식통 - ['20180801142952883']
요청 - ['20180801142859853']
사 - ['201808011451186

## Weighting
- 단어의 빈도 수와 얼마나 적은 문서에서 나왔느냐를 통해서 가중치 계산
- tf: 문서의 개수가 적다면 double normalization K를 이용
- idf: tf는 0과 1 사이이지만 idf는 굉장히 커진다. 그러므로 상용로그 밑수를 무엇을 쓰느냐에 따라 다르게 된다.
- tf가 아무리 크다고 해도 idf의 영향이 커지게 된다.

### TF는 단어의 횟수를 통한 가중치 값이 나온다.

In [12]:
kRatio = 0.5 # 최저 값이 0.5/최고값이 1이 된다. 
TF = {}
for k, v in docNouns.items():
    tfList = {}
    
    for w in v:
        tfList[w] = kRatio + (1-kRatio)*(v[w] / maxCount[k])
#         print("{0} | {1} + {2} * ({3} / {4})  = {5}".format(
#             w, kRatio, (1-kRatio), v[w], maxCount[k], tfList[w])
#         )
    TF[k] = tfList

In [13]:
docNouns

{'20180801141549383': {'인천': 7,
  '인천공항': 3,
  '공항': 5,
  '뉴': 2,
  '뉴시스': 2,
  '시스': 2,
  '임': 1,
  '임태훈': 1,
  '태': 1,
  '훈': 1,
  '기자': 2,
  '저': 6,
  '2': 15,
  '2여객터미널': 3,
  '여객': 6,
  '터미널': 18,
  '개항': 1,
  '첫날': 1,
  '18': 4,
  '18일': 1,
  '일': 3,
  '오후': 2,
  '인천국제공항': 1,
  '국제공항': 2,
  '항공': 12,
  '이륙': 1,
  '2018.1': 1,
  '홍': 1,
  '홍찬선': 1,
  '찬선': 1,
  '델타': 2,
  '에어': 2,
  '에어프랑스': 2,
  '프랑스': 2,
  '네덜란드': 1,
  '사용': 2,
  '중인': 3,
  '올': 3,
  '10': 1,
  '10월': 1,
  '월': 2,
  '7': 5,
  '7개': 4,
  '개': 4,
  '추가': 2,
  '이전': 6,
  '1': 7,
  '1일': 1,
  '인천공항공사': 1,
  '공사': 6,
  '진행': 2,
  '1터미널': 4,
  '시설': 2,
  '재배치': 2,
  '2018': 1,
  '21': 1,
  '21년': 1,
  '년': 2,
  '체크인': 1,
  '체크인카운터': 1,
  '카운터': 1,
  '부족': 1,
  '문제': 1,
  '당초': 1,
  '예측': 1,
  '증가': 2,
  '상황': 1,
  '등': 5,
  '고려': 1,
  '항공사': 13,
  '7곳': 1,
  '곳': 1,
  '올해': 2,
  '동계': 1,
  '동계시즌': 1,
  '시즌': 1,
  '2터미널': 11,
  '결정': 1,
  '에로': 2,
  '에로멕시코': 1,
  '멕시코': 1,
  '리': 1,
  '리딸': 1,
  '딸': 1,
  '중화': 1,
  '중

### IDF 가중치

In [14]:
from math import log2, log10

docSize = len(fileList)
TFIDF = {}

for k,v in TF.items(): 
    # TF에는 각 document 안에 존재하는 단어의 tf값을 dict로 저장 
    idfList = {}
    
    for t in v:
        idf = log10(docSize / len(invertedIndex[t]))
        # 전체 문서 / 단어가 나온 문서 수를 나누어서 로그 값을 확인 
        idfList[t] = v[t] * idf
#         print("{0}|{1} * log10({2}/{3}) = {4}".format(
#             t, v[t], docSize, len(invertedIndex[t]), idfList[t]
#         ))
        
    TFIDF[k] = idfList

In [15]:
TFIDF

{'20180801141549383': {'인천': 0.5403828127664192,
  '인천공항': 0.6295223935277813,
  '공항': 0.6894769071970936,
  '뉴': 0.3344777729599791,
  '뉴시스': 0.3344777729599791,
  '시스': 0.3344777729599791,
  '임': 0.5695678798584687,
  '임태훈': 0.5695678798584687,
  '태': 0.5695678798584687,
  '훈': 0.5695678798584687,
  '기자': 0.02099364493855542,
  '저': 0.5187675002557623,
  '2': 0.1614169874677078,
  '2여객터미널': 0.6295223935277813,
  '여객': 0.7194541640317499,
  '터미널': 1.0791812460476249,
  '개항': 0.5695678798584687,
  '첫날': 0.5695678798584687,
  '18': 0.36792555025597706,
  '18일': 0.5695678798584687,
  '일': 0.022043327185483193,
  '오후': 0.21122846761755892,
  '인천국제공항': 0.5695678798584687,
  '국제공항': 0.5995451366931249,
  '항공': 0.8993177050396873,
  '이륙': 0.5695678798584687,
  '2018.1': 0.5695678798584687,
  '홍': 0.4106909377024786,
  '홍찬선': 0.5695678798584687,
  '찬선': 0.5695678798584687,
  '델타': 0.5995451366931249,
  '에어': 0.43230625021313535,
  '에어프랑스': 0.5995451366931249,
  '프랑스': 0.5995451366931249,
  '네

## Retrieving
- 각 문서의 벡터 크기와 쿼리 와 문서 벡터 간에 거리를 통해서 구할 수 있다
- 쿼리의 거리는 구할 필요가 없다.

In [16]:
query = "자연 재해로 화재가 발생하여 일사병이 발생하였다"

queryNouns = {}
maxCount = 0
        
for word in nltk.word_tokenize(query):
    for w in Kkma().nouns(word):
        if w in queryNouns.keys():
            queryNouns[w] += 1 # 있으면 벡터 값을 더한다.
        else:
            queryNouns[w] = 1 # 없으면 새로운 갑 추가 

    if maxCount < queryNouns[w]:
        maxCount = queryNouns[w]

print(len(queryNouns), maxCount)
print(queryNouns)

4 2
{'자연': 1, '재해': 1, '발생': 2, '일사병': 1}


- IDF document가 하나이기 때문에 tf만 구한 것이다

In [17]:
queryWeight = {}

for k,v in queryNouns.items():
    queryWeight[k] = kRatio + (1-kRatio) * (v/maxCount)
    print("{0} | {1} + {2} * ({3} / {4})  = {5}".format(
                k, kRatio, (1-kRatio), v, maxCount, queryWeight[k])
            )
    

자연 | 0.5 + 0.5 * (1 / 2)  = 0.75
재해 | 0.5 + 0.5 * (1 / 2)  = 0.75
발생 | 0.5 + 0.5 * (2 / 2)  = 1.0
일사병 | 0.5 + 0.5 * (1 / 2)  = 0.75


### Document Length
- 각 문서들의 벡터 크기를 구한다.

In [18]:
from math import sqrt 

docLength = {} # 문서 1: 길이, 문서 2: 길이 ...

for k,v in TFIDF.items(): #k: 문서 ID, v: 단어 1: 가중치, 단어2: 가중치 ...
    sumPow = 0.0
    
    for t in v:
        sumPow += v[t]**2
    
    docLength[k] = sqrt(sumPow)

docLength

{'20180801141549383': 7.796675832536378,
 '20180801144439453': 5.93916075786991,
 '20180801142859853': 5.929019960749014,
 '20180801143528088': 6.3375523108118355,
 '20180801144119329': 8.648300521686817,
 '20180801145118652': 8.039881599116514,
 '20180801143420047': 6.862812425420678,
 '20180801141712421': 6.374369839693533,
 '20180801141138247': 5.14480857635682,
 '20180801145951917': 8.497153109657708,
 '20180801142952883': 8.121564934590152,
 '20180801144437449': 0.0}

In [25]:
queryWeight

{'자연': 0.75, '재해': 0.75, '발생': 1.0, '일사병': 0.75}

In [24]:
result = {}

for k,v in queryWeight.items():
    if k in invertedIndex.keys(): # 각 문서에 그 단어가 있는지 확인한다. 
        print(k)
        
        for docid in invertedIndex[k]:
            if docid in result.keys():
                result[docid] += TFIDF[docid][k] * queryWeight[k] 
                # 각 단어의 가중치 값과 우리가 확인하려는 문서의 가중치 값을 곱한다.
            else:
                result[docid] = TFIDF[docid][k] * queryWeight[k]
            
            print("DocID: {0} - Weight{1}".format(docid, TFIDF[docid][k]))
            # 각 문서에서 가지고 있는 그 단어의 가중치 값을 확인한다. 
            
print()

for k, v in result.items():
    result[k] = v/docLength[k]

result = sorted(result.items(), key=lambda k:k[1], reverse=True) # 내림차순 

# for k, v in result.items():
#     print("[{0}] 유사도: {1}".format(k, v))


for k,v in result:
    print("[{0}] 유사도: {1}".format(k, v))

print()
    
for k, v in result:
    print("{0} - {1}".format(k, v))
    with open('doc/'+ docid+'.txt', 'r') as fp:
        print(fp.readlines(5))


자연
DocID: 20180801144119329 - Weight0.6166749977414999
발생
DocID: 20180801141549383 - Weight0.31775388431198015
DocID: 20180801144119329 - Weight0.5160514211382535
DocID: 20180801145951917 - Weight0.3612359947967774

[20180801144119329] 유사도: 0.11315028507514384
[20180801145951917] 유사도: 0.04251259099782529
[20180801141549383] 유사도: 0.04075504627061171

20180801144119329 - 0.11315028507514384
['\n', '\n', '\n', '\n', '\n', '           【청주=뉴시스】인진연 기자 = 충북지역 낮 최고기온이 38도 이상 오르며 충주와 제천 등에서 역대 최고기온을 경신한 1일, 청주 문암생태공원 나무그늘에서 시민들이 더위를 피하고 있다. 2018.08.01. inphoto@newsis.com\n']
20180801145951917 - 0.04251259099782529
['\n', '\n', '\n', '\n', '\n', '           【청주=뉴시스】인진연 기자 = 충북지역 낮 최고기온이 38도 이상 오르며 충주와 제천 등에서 역대 최고기온을 경신한 1일, 청주 문암생태공원 나무그늘에서 시민들이 더위를 피하고 있다. 2018.08.01. inphoto@newsis.com\n']
20180801141549383 - 0.04075504627061171
['\n', '\n', '\n', '\n', '\n', '           【청주=뉴시스】인진연 기자 = 충북지역 낮 최고기온이 38도 이상 오르며 충주와 제천 등에서 역대 최고기온을 경신한 1일, 청주 문암생태공원 나무그늘에서 시민들이 더위를 피하고 있다. 2018.08.01. inphoto@